### Introduction

- 위키피디아, 이안 굿펠로우, 벤지오 교수님의 오토인코더의 설명에서의 키워드를 살펴보면 다음과 같다.

- 오토인코더를 학습할 때:
    - 1. **Unsupervised learning** (학습 방법은 비교사 학습 방법을 따르며,)
    - 2. **ML density estimation** (Loss는 negative ML로 해석된다.)
    
- 학습된 오토인코더에서:
    - 3. **Manifold learning** (인코더는 차원 축소 역할을 수행하며,)
    - 4. **Generative model learning** (디코더는 생성 모델의 역할을 수행한다.)
     
     
## Ch 1. Revisit DNN

- **DNN이 학습할 때 MLE를 하는 것임을 알아가자.**

- DNN의 Loss function은 보통 MSE, Cross Entropy 였다.
    - 그럴 수 밖에 없는게 backpropa 때문이다.
    - Backpropa의 가정:
    - a. Total loss of DNN over training samples is the sum of loss for each training sample. (개개의 데이터의 loss와 합은 같다)
    - b. Loss for each training example is a function of final output of DNN. (출력값과 정답만 가지고 구성하겠다. 중간이 아니라 googlenet은 중간에 걸 사용하는 것 같아 보이지만 종단이라 생각하고 뒤로 백프로파한다.)

- DNN가 파라미터를 추정하는 방법은 대부분 Gradint Descent였다.
    - Iterative한 방법으로 속도와 언제 멈출 것인지를 정한다.
    - 방향을 정할 때, 테일러 확장을 통해 1차미분만 사용하는 것은 현재 세타값의 근처만 보겠다는 것을 의미한다.
    - <img  src="./img/AAA-01.PNG" width="90%">
    - 2번 째 줄을 넘겨 3번째 줄을 만들고 delta L 이 음수가 되고 싶은데
    - 델타 세타를 -gradient로 하면 항상 음수가 되니까 여기에 예타(lr)를 붙여서 사용한다.

- 이를 종합하고 SGD로 바꾸어 표현하고 백프로파를 적용하면,
    - <img  src="./img/AAA-02.PNG" width="70%">
    - 막단의 error signal을 구하고 차례대로(delta rule로) signal을 구하면, bias는 error signal 그 자체가 되고, Weight은 layer의 입력을 곱하면 구할 수 있다. [상세설명]( http://neuralnetworksanddeeplearning.com/chap2.html)
    

- 여기서, 아까 말한 Loss Func.인 MSE와 Cross entropy중 무엇이 좋은지 Golden Rule이 있는지? 이 때 2개의 관점이 있다.
    - **1. Backpropa 알고리즘이 잘 동작하는 관점**에서는 Cross entropy가 MSE가 좋다고 볼 수 있다. 
    - **2. MLE의 관점**에서는 output이 연속일 때는 MSE, discrete할 때는 Cross entropy 게 맞다는 해석이 있다.
    
###  Loss function viewpoints I (Backpropation View)

- MSE를 할 때: 1개의 layer에서 NN에서 MSE Loss로 random하고 초기값이 다른 2 개의 300 iter에서 멈추고 결과를 보면 초기값 차이가 큰 차이를 보인다.
    - 이유는 gradient에 activation의 미분값(sigmoid)이 계속해서 곱해지니까.
    - gradient vanishing 그래서 Relu가 그나마 덜 취약.

- CE를 할 때: 는 절묘하게 에러값에 activation function의 미분값이 곱해지지 않아(막 레이어만) gradient vanishing problem에서 좀 더 자유롭다. 학습이 좀 더 빨리 된다. 근데 막단에서만 사라지는 거여서 너무 깊으면 완전히 자유롭진 않지만 시작할 때부터 1/4이 줄어드는 것은 큰 차이일 수도 있다.
    - <img  src="./img/AAA-03.PNG" width="90%">
    - 파: MSE  /  빨: CE
    
### View-Point II : Maximum Likelihood

- 네트워크의 출력값(y', $f_{\theta}(x)$)가 주어질 때, y가 나오는 확률이 크길 바란다.
    - $p(y|f_{\theta}(x))$
    - MLE의 관점에서는 정할게 한게 더있다고 생각 할 수 있다.
    - 예를 들어 $p(y|f_{\theta}(x))$ 이것이 가우시안, 베르누이 ... 등을 따를 것이다. 로 해석할 수 있다. 가우시안이라고 생각하고 평균 만 생각해 보면
    - <img  src="./img/AAA-04.PNG" width="40%">
    - 그리고 이때 Loss는 $ -log(p(y|f_{\theta}(x))) $ 가 되며, 우리는 확률 분포 모델을 찾았으므로(모델링 했으므로) sampling이 가능하다.
    - $y_{new} ~ p(y|f_{\theta}^{*}(x_{new})) $
    - 이 관점으로 해석해야 VAE를 이해하기 슆다/ 개를 학습하면 서로 다른 개 그림이 나오고 개를 잘 이해했다고 보는 것. 물론 ML Estimation으로 보면은 평균이랑 같아야하므로 고정 출력으로 보는 것이다.
    - 그리고 $p(y|f_{\theta}(x))$ 에 IID condition을 적용하면, 위의 backpropa를 하기 위한 가정을 만족한다.


- 그리고 확률 분포 모델을 가우시안과 베르누이라고 가정하고 식을 따라가 보면,
    - 가우시안은 MSE, 베르누이는 CE의 수식이 나온다.
    - 그래서 ML의 관점에서 우리가 구해야할 확률 분포가 가우시안 분포에 가까우면 MSE 베르누이 분포에 가까우면 베르누이를 쓰는 것이 좋으므로 연속적인 값은 MSE, 이산인 값은 베르누이를 따르는 것이 좋다고 해석 할 수 있다. (다변수여도, 다른 확률 분포여도 식을 만족한다.)
    - <img  src="./img/AAA-05.PNG" width="100%">

- ML의 관점에서 해석할 때, Likelihood값을 예측하는 것(네트워크의 출력값)이 아니라, 확률분포를 정하고 그 확률 분포를 규정짓는 파라미터를 추정한 것이다. (Likelihood의 파라미터값을 예측하는 것이다. 가우시안에서는 평균(분산도 가능), 베르누이의 확률값을 추정하는 것.)
    -  <img  src="./img/AAA-06.PNG" width="100%">

- 그리고 지금까지 배운 것을 벤지오교수님은 한 큐에 설명했다.
    -  <img  src="./img/AAA-07.PNG" width="100%">

- **오토인코더는 $ p(x|x) $를 추정**하는 것이고 **VAE는 $ p(x) $ 확률분포를 추정**하는 것이다.


## Ch 2. Manifold Learning
- 50분쯤
- Autoencoder의 가장 중요한 기능 중 하나는 매니폴드를 학습한다는 것
- 매니폴드 학습의 목적 4가지인 데이터 압축, 데이터 시각화, 차원의 저주 피하기, 유용한 특징 추출하기에 대해서 설명
- Autoencoder의 주요 기능인 차원 축소, 확률 분포 예측과 관련된 기존의 방법들을 살펴보고, 그 한계점에 대해서 짚어보자.


- Manifold란
    - 고차원을 3차원이라 하고, 저차원을 2차원이라할 때, 위의 3차원 그림 처럼 데이터를 잘 아우르는 subspace이고 이를 잘 찾으면, 차원축소도 잘 될 것이다.

- 목적: 
    - 1. Data compression
    - 2. Data visualization
    - 3. Curse of dimensionality
        - Manifold Hypothesis
    - 4. Discovering most important features
        - Reasonable distance metric
        - Needs disentagling the underlying explanatory factors (making sense of the data)
        - MPEG, CODEC 또 이런 기능을 한다.
    - 3,4번이 중요해 보인다.


### a. Data compression:

- Lossy Image Compression with compressive Autoencoders, ‘17.03.01'
- JPEG 보다 압축이 잘 되더라. 이런건 많다.


### b. Data visualization:

- 직관, 결과물에 대한 해석. 포장은 중요하다.
- http://vision-explorer.reactive.ai/#/?_k=aodf68
- https://fontjoy.com/projector/


### c. Curse of dimensionality

- 차원의 저주: 데이터의 차원이 증가할수록 해당 공간의 크기(부피)가 기하급수적으로 증가하기 때문에 동일한 개수의 데이터의 밀도는 차원이 증가할수록 급속도로 희박해짐.
- 따라서, 차원이 증가할수록 데이터의 분포 분석 또는 모델추정에 필요한 샘플 데이터의 개수가 기하급수적으로 증가하게 됨.
    
- Manifold Hypothesis (assumption) 
    - Natural data in high dimensional spaces concentrates close to lower dimensional manifolds.Probability density decreases very rapidly when moving away from the supporting manifold.
    - 고차원의 데이터의 밀도는 낮지만, 이들의 집합을 포함하는 저차원의 매니폴드가 있다. 이 저차원의 매니폴드를 벗어나는 순간 급격히 밀도는 낮아진다.
    
- 이를 느끼기 위해 200 x 200 RGB에서는 10^96329 possible states가 있는데 엄청많이 아무리 random sample 해도 노이즈밖에 안나온다.
    - 200 x 200 RGB에서 거의 모든 그림을 다 표현할 수 있다.
    - 그래서 Manifold를 잘 학습하는 GAN과 같은 것은 이제 1000 x 1000 RGB 까지는 꽤 Manifold를 잘 학습한다.
    - 모든 이미지는 잘 안되지만 1000개 클래스(이미지넷) 정도는 잘되는 수준이다.


### d.  Discovering most important features

- Manifold follows naturally from continuous underlying factors (≈intrinsic manifold coordinates) Such continuous factors are part of a meaningful representation!
    - 매니폴드 학습 결과 평가를 위해 매니폴드 좌표들이 조금씩 변할 때 원 데이터도 유의미하게 조금씩 변함을 보인다.
    - <img  src="./img/AAA-08.PNG" width="70%">
    - rotation, thickness와 같은 의미가 자동으로 찾아지는 것이다. (Unsupervised이기 때문에)
    - 그리고 이런 것이 잘 찾아져야 좋은 압축이라 할 수 있다.

- Reasonable distance metric
    - 의미적으로 가깝다고 생각되는 고차원 공간에서의 두 샘플들 간의 거리는 먼 경우가 많다. 고차원 공간에서 가까운 두 샘플들은 의미적으로는 굉장히 다를 수 있다. 차원의 저주로 인해 고차원에서의 유의미한 거리 측정 방식을 찾기 어렵다.
    - 그런데 중요한 특징을 찾았다면 잘 찾아야한다.
    - <img  src="./img/AAA-09.PNG" width="70%">
    - Manifold가 아닌 공간에서 유클리디안 거리에서는 그냥 겹쳐진 사진이 보여짐.
    - 이를 다르게  Entangled manifold, Disentangled(혼란스러운 주장·생각 등을) 구분하다) manifold이라는 말로도 표현한다.


### Dim. Reduction

- Dimensionality Reduction:
    - Linear:
        - Principal Component Analysis (PCA)
        - Linear Discriminant Analysis (LDA) etc..
    - Non-Linear:
        - Autoencoders (AE)
        - t-distributed stochastic neighbor embedding (t-SNE)
        - Isomap
        - Locally-linear embedding (LLE) etc..

- AE 말고 차원축소를 하는 알고리즘은 많다. 기존 ML의 차원축소를 보고 AE와 무엇이 다르른지 보자.


- a. PCA: 아이젠 벡터가 큰 (분산이 최대인 축)으로 차원을 축소.
    - projection 하는 수식: $ h = f_{\theta}(x) = W(x - \mu) $ with $ \theta = \left\{ W, \mu \right\} $
    - 을 보면, 평균을 빼서 W를 고유벡터값이 큰 순서대로 구성해서 만든다.
    - 근데 식을 보면 NN의 form ($ Wx + b$)과 비슷하다. 뒤에 말하겠지만 오토인코더가 PCA를 포함하는 방법론임이 80년도에 증명이 되어있다.
    - 하지만 스위스 롤과 같은 데이터는 linear 방법으로 차원축소가 잘 되지 않는다. (Entangled 하게 축소한다.)
    
- b. Non linear methods: 
    - Isomap과 LLE의 첫 과정만 알아 보면, KNN을 통해 근방을 구하는 작업을 먼저하는데 그 이유는 이정도 가까운 애들은 manifold의 상에서도 가까울 거라는 가정에서 시작한다.


### Density Estimation

- Parzen Windows
    - 1D 샘플에서 샘플별로 가우시안 커널을 씌워서, 더한 것으로 확률 분포를 모델링하는 방식. 이것도 점점 성능이 발전한다.
    - Manifold Parzen Windows(Vincent and Bengio. NIPS, 2003): locally PCA를 돌려서 방향성을 고려해 covar를 구한다.
    - Non-local Manifold Parzen Windows(Bengio et al. NIPS, 2006): 평균도 고려해서 하니까 더 잘되더라


- Density Estimation, Dim. Reduction 의 기존의 방법들은 Neigborhood based training이었다.
    - But in high d, your nearest Euclidean neighbor can be very different from you.
    - 위에서 말한 것처럼, 고차원 데이터 간의 유클리디안 거리는 유의미한 거리 개념이 아닐 가능성이 높다. (이웃이 이웃이 아닐 수 있다.)
    - 결론적으로 DNN 오토인코더는 언제 더 좋은가. 경계는 없지만 Data가 많고, 고차원 데이터일 수록 좋다고 할 수 있다.



## Ch 3. Autoencoders

- Terminology
    - Autoencoders = Auto-associators = Diabolo networks = Sandglass-shaped net
    - <img  src="./img/AAA-10.PNG" width="60%">    
    - Dim reduction의 관점에서 AE가 Unsupervised를 Supervised로 바꾸어 문제를 풀었기 때문에 각광을 받았었다. (Supervised라고도 하지만 Self Learning이라는 말도 사용
    - 그리고 학습이 끊나고 사용할 때는 인코더와 디코더를 따로 사용하는데
    - 인코더는 최소한 학습 데이터는 latent vector로 잘 표현한다는 것을 보장해주고
    - 디코더는 최소한 학습 데이터는 만들어 낼 수 있다. GAN은 이것을 보장해주지 않아 학습이 잘 안되긴 하지만, 대신에 Generation에서 다양성은 GAN이 더 좋음.
    

- General AE 와 Linear AE의 차이는 Activation function이다.
    - Linear AE를 학습하면 PCA와 basis는 다르지만, 같은 manifold를 학습한다고 1989 논문에 나왔다.


- Stacking AE: AE가 처음에는 Batch Norm, xavier init. 과 같은 것이 없었을 때에는 파라미터를 초기화같은 Pre-Training 할 때 많이 사용되었다.
    - layer by layer로 적어도 복원할 수 있는 값으로 초기화. 지금은 사용하지 않는다.
    

- Denoising AE: 
    - 노이즈를 넣은 값을 Encoder에 태우고 Decoder에서 나온 값과 비교할 때 노이즈 안나온 값이 나오도록 Loss를 구성한 AE
    - 노이즈는 사람이 구분할 수 있을 정도로. 음성 '아' 를 넣으면 적어도 '아'라고는 들리게
    - 그러면 manifold를 학습할 때 깨끗한 '아'와 같은 noise 있는 '아'가 의미적으로 같은 manifold 공간에 있어야 해야되어서 좀 더 manifold가 잘 배워진다고 했다.
    - <img  src="./img/AAA-11.PNG" width="70%"> 
    - 그리고 low level feature에서 noise 처럼 보이지 않고 의미를 갖는 선과 같이 나오면 학습이 잘됬다고 할 수 있는 데 결과를 보면 잘 되었다.
    

- Stacked Denoising AE: 이 때도 Noise를 어느 정도 추가했을 때 더 좋았고 generation도 더 잘 했다.


- Stochastic Contractive AE: 
    - 기존의 AE의 reconstruction error에서는 noise를 regularizer로 넣었다.
    - $ {L_{SCAE} = \sum_{x \in D}} {\color{Blue}L(x,g(h(x)))} + {\lambda} 
    {\color{Red}{E_{q(\tilde{x} | x)}\left[||h(x)-h(\tilde{x})||^{2}\right]}}  $
    - Contractive AE는 stochastic regularization term을 stochastic하게 구한 것이 아니라 deterministic하게 구하고 계산하는 것.
    - 근데 거의 CAE까지 잘 안쓰고 AE, DAE, VAE를 요새 사용한다.



## Reference

- https://tv.naver.com/v/3185672?query=%EC%98%A4%ED%86%A0%EC%9D%B8%EC%BD%94%EB%8D%94%EC%9D%98+%EB%AA%A8%EB%93%A0%EA%B2%83&plClips=false:3185672:3185730:3186249
- https://www.slideshare.net/NaverEngineering/ss-96581209